In [1]:
!pip install torch==1.10.0 torchvision==0.11.1 torchtext==0.11.0 torchaudio==0.10.0 --quiet
!pip install pytorch-lightning==1.5.10 --quiet
!pip install lightning-flash==0.7.1 --quiet
!pip install 'lightning-flash[audio,image, video, text]' --quiet
!pip install Pillow==9.0.0


     |██████████████████████████████▎ | 834.1 MB 1.4 MB/s eta 0:00:34tcmalloc: large alloc 1147494400 bytes == 0x21ba000 @  0x7fec42edf615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████████████████████| 881.9 MB 20 kB/s 
     |████████████████████████████████| 23.3 MB 33.4 MB/s 
     |████████████████████████████████| 8.0 MB 35.9 MB/s 
     |████████████████████████████████| 2.9 MB 4.9 MB/s 
     |████████████████████████████████| 527 kB 5.2 MB/s 
     |████████████████████████████████| 596 kB 28.3 MB/s 
     |████████████████████████████████| 409 kB 44.6 MB/s 
     |████████████████████████████████| 136 kB 51.9 MB/s 
     |████████████████████████████████| 829 kB 41.3 MB/s 
     |████████████████████████████████| 952 kB 30.6 MB/s 
     |██████

In [2]:
import pandas as pd
import random
import torch
import flash

from sklearn.model_selection import train_test_split
from flash import Trainer
from flash.audio import SpeechRecognitionData, SpeechRecognition

In [3]:
print("pandas version:",pd.__version__)
print("torch version:",torch.__version__)
print("flash version:",flash.__version__)


pandas version: 1.3.5
torch version: 1.10.0+cu102
flash version: 0.7.1


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


###Please follow instructions in the book to get dataset and put into your google drive.

In [ ]:
!unzip '/content/gdrive/My Drive/Colab Notebooks/scottish_english_female.zip'

In [ ]:
random.seed(10)
df_scottish = pd.read_csv("line_index.csv", header=None, names=['not_required', 'speech_files', 'targets'])
df_scottish = df_scottish.sample(frac=0.06)
print(df_scottish.shape)
df_scottish.head()

(54, 3)


,not_required,speech_files,targets
802,EN0007,scf_05223_01592871455,Throughout the centuries people have explaine...
807,EN0003,scf_05223_00004260276,These take the shape of a long round arch wit...
548,EN0120,scf_07049_01518194427,Since you're not checking notifications from ...
429,EN0036,scf_07049_01731861458,Direct actions employ specialized military ca...
56,EN0475,scf_04310_01632466472,The Google assistant can work with Google par...


In [ ]:
df_scottish = df_scottish[['speech_files', 'targets']]
df_scottish['speech_files'] = df_scottish['speech_files'].str.lstrip()

df_scottish['speech_files'] = df_scottish['speech_files'].astype(str) + '.wav'
df_scottish.head()

,speech_files,targets
802,scf_05223_01592871455.wav,Throughout the centuries people have explaine...
807,scf_05223_00004260276.wav,These take the shape of a long round arch wit...
548,scf_07049_01518194427.wav,Since you're not checking notifications from ...
429,scf_07049_01731861458.wav,Direct actions employ specialized military ca...
56,scf_04310_01632466472.wav,The Google assistant can work with Google par...


In [ ]:
random.seed(10)
train_scottish, test_scottish_raw = train_test_split(df_scottish, test_size=0.2)
test_scottish = test_scottish_raw['speech_files']
test_scottish.head()

310    scf_03397_01072622319.wav
223    scf_07049_01133867565.wav
868    scf_04310_00395537459.wav
835    scf_04310_02050219481.wav
620    scf_07049_00239785467.wav
Name: speech_files, dtype: object

In [ ]:
train_scottish.to_csv('train_scottish.csv')
test_scottish.to_csv('test_scottish.csv')

In [ ]:
datamodule = SpeechRecognitionData.from_csv(
     "speech_files",
     "targets",
     train_file="train_scottish.csv",
     predict_file="test_scottish.csv",
     batch_size=10)

Using custom data configuration default-15b9ea74ae447d33


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-15b9ea74ae447d33/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-bfc3565a6d97d227


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-bfc3565a6d97d227/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
SpeechRecognition.available_backbones()

['Anything available from: Hugging Face/transformers (https://github.com/huggingface/transformers)',
 'facebook/wav2vec2-base-960h',
 'facebook/wav2vec2-large-960h-lv60']

In [ ]:
model = SpeechRecognition(backbone="facebook/wav2vec2-base-960h")

Using 'facebook/wav2vec2-base-960h' provided by Hugging Face/transformers (https://github.com/huggingface/transformers) and PyTorch/fairseq (https://github.com/pytorch/fairseq).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print(model.get_backbone_details("facebook/wav2vec2-base-960h"))

[('model_args', <Parameter "*model_args">), ('kwargs', <Parameter "**kwargs">)]


In [ ]:
trainer = Trainer(max_epochs=4, gpus=-1, precision=16)
trainer.finetune(model, datamodule=datamodule, strategy="no_freeze")


Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type           | Params
-------------------------------------------------
0 | model         | Wav2Vec2ForCTC | 94.4 M
1 | train_metrics | ModuleDict     | 0     
2 | val_metrics   | ModuleDict     | 0     
3 | test_metrics  | ModuleDict     | 0     
-------------------------------------------------
94.4 M    Trainable params
0         Non-trainable params
94.4 M    Total params
188.793   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

In [ ]:
trainer.predict(model, datamodule=datamodule)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 4it [00:00, ?it/s]

[['CREDITCARD COMPANIES PURCHASE CHARGED OFF DEBTS IDN ADD THEM TO THE BALANCE OF ABAT AND SWITCH CREDIT CARDS',
  'THE JAR CAN ONLY HOLD UP TO TANE MARBLES',
  'SMALL SQUARE IMAGE IS REQUIRED',
  'JUST TOUCH AND HOLD THE RIGHT EAR BUD AND START TALKING',
  'THE POWERS OF THE APPOINTED CHEER ARE LIMITED SO THAT THE CHEER CANNOT ADJOURN A MEETING AT ANY POINT WITHOUT THE MAJORITY VOTE',
  'THE SOLEMNITY OF THE OCCASION REQUIRED ATTENTION AND RESPECT',
  "VOICE MATCH HELPS THE ASSISTANT IDENTIFY HEATHER'S VOICE AND TELL HER APART FROM OTHERS BY CREATING A UNIQUE MODEL OF HER VOICE ON THIS DEVICE",
  "THE GIGGLE ASSISTANT CAN WORK WITH GUGGL PARTNERS TO HELP YE GET THINGS DONE WHETHER IT'S BANG MOVY TICKETS OR GETING YOUR SINK REPAIRED",
  'ON CERTAIN OPERATING SYSTEMS FOWLS MUST BE MOVED OUT OF THE TRASH BEFORE THEY CAN BE ACCESSED AGAIN',
  'CONSUMER CONFUSION IS A STATE OF MIND THAT LEADS TO CONSUMERS MEEKING IMPERFECT PURCHASING DECISIONS'],
 ['CONCENTRATED SOILER POWER USES MOLTEN SA

In [ ]:
train_scottish.head()

,speech_files,targets
806,scf_04310_00982287028.wav,Genre's foot soldiers think that literary fic...
317,scf_02484_00826871932.wav,Alice was default reading to the point where ...
830,scf_02484_00532773368.wav,A single draught horse can normally pull a si...
157,scf_03397_00005905407.wav,According to the Freudian model the ego is th...
351,scf_05223_01725020876.wav,Most of the countries that border the Persian...


In [ ]:
datamodule = SpeechRecognitionData.from_files(predict_files='scf_02484_00097285334.wav', batch_size=10)

predictions = trainer.predict(model, datamodule=datamodule)
print(predictions)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 4it [00:00, ?it/s]

[['HARRY THEREAFTER RAISED SALARIES FORTUDU EMPLOYES']]


In [ ]:
test_scottish_raw.head()

,speech_files,targets
310,scf_03397_01072622319.wav,Credit card companies purchase charged-off de...
223,scf_07049_01133867565.wav,The jar can only hold up to 10 marbles
868,scf_04310_00395537459.wav,Small square image is required
835,scf_04310_02050219481.wav,Just touch and hold the right earbud and star...
620,scf_07049_00239785467.wav,The powers of the appointed chair are limited...


In [ ]:
datamodule = SpeechRecognitionData.from_files(predict_files='scf_02484_01925002707.wav', batch_size=10) # Scottish

predictions = trainer.predict(model, datamodule=datamodule)
print(predictions)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 4it [00:00, ?it/s]

[['ASK HER TO CAREFULLY BRING THESE THINGS WITH HER FROM THE STORE']]
